<a href="https://colab.research.google.com/github/nicolaiberk/nlpdl_project/blob/main/Kopie_von_BaselineBERT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

# all code is based on https://huggingface.co/transformers/custom_datasets.html#seq-imdb

In [ ]:
# define model
from transformers import BertForSequenceClassification
checkpoint_model = 'distilbert-base-german-cased'
model = BertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

In [ ]:
# define optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
# define tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(checkpoint_model)

In [ ]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/germanyPPRs.csv', engine="python")
df = df.reset_index()
df = df.dropna()
df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [ ]:
# test data: IMDP from Huggingface tutorial

!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [ ]:
# train/test split
trainset = df[df.date < pd.to_datetime('2018-01-01', format='%Y-%m-%d')]
testset = df[df.date >= pd.to_datetime('2018-01-01', format='%Y-%m-%d')]

In [ ]:
# validation split
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(trainset['rawtext'], trainset['label'], test_size=.2) # german PPRS
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2) # IMDB

In [ ]:
# tokenize
train_encodings = tokenizer([x for x in train_texts], truncation=True, padding=True, max_length=512)
val_encodings = tokenizer([x for x in val_texts], truncation=True, padding=True, max_length=512)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512) #IMDB
test_encodings = tokenizer([x for x in testset['rawtext']], truncation=True, padding=True, max_length=512) # PPRs
test_labels = [x for x in testset['label']] # PPRs

In [ ]:
# turn labels and encodings in dataset object
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
# fine-tune BERT

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

model = BertForSequenceClassification.from_pretrained(checkpoint_model)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

KeyError: ignored